# Writing Efficient Python Code

## Foundations for efficiencies
- Efficient Python code has fast runtime (low latency) and minimal resource consumption (small overhead).
- Use NumPy arrays to broadcast operations. Other advantages: indexing and boolean masking.
- Use built-in function map more.

**Unpacking** with **range** and **enumerate**

In [2]:
[*range(1, 11, 1)]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [3]:
names = ['John', 'Mike', 'Brenda']
[*enumerate(names)]

[(0, 'John'), (1, 'Mike'), (2, 'Brenda')]

## Timing and Profiling Code

**IPython Magic commands**: prefixed with % (%lsmagic lists all available magics)  

**%timeit** (line) / **%%timeit**(cell)
- runs: how many iterations to estimate the run time (-r)
- loops: number of executions per run (-n)
- save time in a variable (-o)

**Performance**
- Unpacking can be faster than list comprehension ([*range(1,10)])
- Using literals for datatypes is faster ([] instead of list())
- Broadcasting with numpy is also faster

In [14]:
times = %timeit -r 10 -n 1000 -o a=1

14.2 ns ± 0.098 ns per loop (mean ± std. dev. of 10 runs, 1000 loops each)


In [22]:
vars(times).keys()

dict_keys(['loops', 'repeat', 'best', 'worst', 'all_runs', 'compile_time', '_precision', 'timings'])

In [23]:
%%timeit
nums = []
for x in range(10):
    nums.append(x)

862 ns ± 76 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


**Code Profiling / line_profiler package**  
Detailed statistics on frequency and duration of function calls / Line-by-line analysis  

**%lprun**
- -f: profile a function

In [2]:
%load_ext line_profiler
# magic command for line-by-line times
%lprun -f function_name function_call(arguments)

**Code profiling for memory**   
Using sys for the number of bytes:

In [1]:
import sys
num_list = [*range(1000)]
sys.getsizeof(num_list)

9112

Better way: using **memory_profiler** package
- functions need to be defined in separate files
- not as reliable as time profiling

In [ ]:
%load_ext memory_profiler
from module import function_name
%mprun -f function_name function_call(arguments)

## Gaining Efficiencies
- Use NumPy + Broadcasting, list comprehensions and/or map, and holistic datatype conversion

In [3]:
# zip with unpacking
names = ['Jonh', 'Mary', 'Amanda']
ages = [25, 65, 40]

[*zip(names, ages)]

[('Jonh', 25), ('Mary', 65), ('Amanda', 40)]

**collections** module
- Counter: dict for counting hashable objects
- defauldict: dict that calls a factory function to supply missing values

In [24]:
# faster than using
from collections import Counter

objects = ['Grass', 'Grass', 'Fire', 'Water']
type_counts = Counter(objects)
type_counts

Counter({'Grass': 2, 'Fire': 1, 'Water': 1})

In [21]:
from collections import defaultdict

elements = defaultdict(lambda:'Not Available')
elements['John'] = 25
elements['Mary']

'Not Available'

**itertools** module
- Combination generators: product, permutations, combinations

In [27]:
# combinations (order does not matter and no repeated elements)
from itertools import combinations

types = ['High', 'Medium', 'Low']

comb_objs = combinations(types, 2)
[*comb_objs]

[('High', 'Medium'), ('High', 'Low'), ('Medium', 'Low')]

**Set Theory**  
Python has built-in set datatype and methods
- intersection(): elements in both sets
- difference(): all elements in one set but not the other
- symmetric_difference(): all elements in exactly one set
- union(): duplicates are gathered once
- membership checking with **in** operator (faster using sets)
- sets: good way to find unique values

In [31]:
# much faster than using loops
list_a = [5, 10, 15, 20]
list_b = [0, 10, 20]

set_a = set(list_a)
set_b = set(list_b)

inter = set_a.intersection(set_b)
10 in inter

True

## Basic Pandas Optimizations

In [41]:
import pandas as pd
from sklearn.datasets import load_boston

boston = load_boston()
boston_df = pd.DataFrame(boston['data'], columns=boston['feature_names'])

**.iterrows()**  
Not very good performance (better using vectorization)

In [54]:
tax_age = []

for i, row in boston_df.iterrows():
    tax_age.append(row['TAX']/row['AGE'])

**.itertuples()**    
Better performance than .iterrrows()  
Attention: named tuples do not support brackets. We must use dot notation like below:

In [52]:
tax_age = []

for row_namedtuple in boston_df.itertuples():
    tax_age.append(row_namedtuple.TAX/row_namedtuple.AGE)

**.apply()**  
Faster than but approaches above, but slower than vectorization
Specify 0 for columns or 1 for rows  
It can be slow if executed with a function that requires native Python:
https://stackoverflow.com/questions/24870953/does-pandas-iterrows-have-performance-issues

In [60]:
boston_df.apply(lambda row: row['TAX']/row['AGE'], axis=1)

0      4.539877
1      3.067174
2      3.960720
3      4.847162
4      4.095941
         ...   
501    3.950796
502    3.559322
503    3.000000
504    3.057111
505    3.378713
Length: 506, dtype: float64

**vectorized operations**  
MUCH faster thant the other approaches

In [64]:
boston_df['TAX']/boston_df['AGE']

0      4.539877
1      3.067174
2      3.960720
3      4.847162
4      4.095941
         ...   
501    3.950796
502    3.559322
503    3.000000
504    3.057111
505    3.378713
Length: 506, dtype: float64